In [2]:
!ls

input.txt  nGPT.ipynb  training.py


In [1]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1115394


In [5]:
# let's look at the first 1000 characters
print(text[:500])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor


In [7]:
# unique chars
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [20]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("my name is Alex"))
print(encode("hii"))
print(decode(encode("hii")))

[51, 63, 1, 52, 39, 51, 43, 1, 47, 57, 1, 13, 50, 43, 62]
[46, 47, 47]
hii


In [23]:
# let's now encode the entire text dataset and store it into a torch.Tensor
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:500]) # here's the 1000 characters

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [25]:
n = int(0.9*len(data))
training_data = data[:n]
val_data = data[n:]

In [27]:
block_size = 8
training_data[:block_size+1] # inclding the 8th index

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [28]:
x = training_data[:block_size]
y = training_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target is: {target}")

when input is tensor([18]) the target is: 47
when input is tensor([18, 47]) the target is: 56
when input is tensor([18, 47, 56]) the target is: 57
when input is tensor([18, 47, 56, 57]) the target is: 58
when input is tensor([18, 47, 56, 57, 58]) the target is: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target is: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target is: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target is: 58


In [ ]:
torch_manual_seed=(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = training_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,)) # size of {batch_size} - random offsets in the train set
    x = torch.stack([data[i:i+block_size] for i in ix]) # in rows of 4,8da
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

In [31]:
# feeding into the transformer/neural network

In [32]:
print(xb)

tensor([[44, 39, 58, 46, 43, 56,  6,  0],
        [ 1, 53, 56,  1, 39,  1, 58, 59],
        [ 6,  1, 57, 47, 56,  6,  1, 46],
        [56,  1, 41, 39, 59, 57, 43,  1]])


In [54]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step(  seq length)
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample 1 index per sequence from the batch distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # The sampled index (idx_next) is appended to the running sequence (idx) along the 
            # 2nd dimension (dim=1). This operation extends the sequence by one token for each 
            # sequence in the batch. After this step, idx has the shape (B, T+1).
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.6236, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [60]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

batch_size = 64
for steps in range(20000):
    xb, yb = get_batch('train') # sample a batch of data
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.4350619316101074


In [62]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=1000)[0].tolist()))


ment bowhelir s, e ike yors torel deentheaud al hy m co che d Thayem.
Fre. made y EORENo f l toungomys ipllceners fout?
ds.
A: nl an e s REDake t thed hext, have, f thent, udome Ithed Thectchellomougrouspalee

TItan:
eresors w s f by.
Thand.
HALo y wa herem? mer alle:
BENEn m an,
to wilomath my n asoue RII t, ug the weind d avoone ithoucho igmeis OMPlk'selloube wink ngende,

'd y, Hit s,
Ande,
HE:
A feveasw blu nthes isimy y asul s, ws te, ras ty Weiscea as ilthan soorscho.
bucoyotitrcitrat I my HN:

I hatsthouoondivincerisar ben benos, w hy k, RAUKe dyon the, akes:

I te leanckimit, HA wald be IOFisowse t pucomies my:
Thithr y tal ale'stin

STheckemisayoin hefe.
The my totortrar rthooul t: te blale;
DUTh ld, s,


GLed s afovinstenyor ssit serpeto,
ICoranir tim out hime t,


AUpin



HAnewir stecond, stis om.

Bee athin winksur frowas tht cuthasoinckselllat sle te berss he.
ABY:
He s, west he ethangeres, y mu trlin alss

We prbeat bonds prits, otis ty yoome alele my
laveare thy ld,
RI